# Raster data

## Some basic principles

> a raster graphic represents a two-dimensional picture as a rectangular matrix or grid of ~square pixels~
<div>
<img src="../examples/simple_raster.png" width="500"/>
</div>

source: [wikipedia](https://en.wikipedia.org/wiki/Raster_graphics#Image_storage)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
raster = np.random.rand(20, 20)
plt.imshow(raster, cmap="gray")

A simplistic representation of an image is just an array of shape `x, y, 3` where the last axis represents the red, green and blue chanels.

In [ ]:
image = np.random.rand(5, 5, 3)
plt.imshow(image)

In [ ]:
_, axs = plt.subplots(1, 3)
axs[0].imshow(image[:, :, 0], cmap="Reds")
axs[1].imshow(image[:, :, 1], cmap="Greens")
axs[2].imshow(image[:, :, 2], cmap="Blues")

In [ ]:
plt.imsave("../data/example.png", image)

## But...  maps?

Rasters can be _georeferenced_ to represent data on earth surface. 

In order to make it work we need to know
- where the raster is on earth
- the size of the pixels
- which reference system we are using: the Coordinate Reference System (CRS)

All this adds new metadata to our simple image format.

Here is where we will take advantage of libraries like [Rasterio](https://rasterio.readthedocs.io/en/latest/index.html).

Let's try to convert our image to fully privileged geospatial raster

In [ ]:
import rasterio as rio
from rasterio.transform import Affine
from rasterio.plot import reshape_as_raster, reshape_as_image

# we are going to use a lat long projection so units must be in degrees
# as a proj4 string
crs = "+proj=latlong"
# crs can also be defined using the espg code
# crs = "epsg:4326"

# note it is negative! we are defining things from the upper left corner.
xres, yres = 0.0010, -0.0010

transform = Affine.translation(2.944591, 39.445561) * Affine.scale(xres, yres)

with rio.open(
    "../data/raster.tiff",
    mode="w",
    height=image.shape[0],
    width=image.shape[1],
    count=3,  # number of band
    dtype=image.dtype,
    crs=crs,
    transform=transform,
) as new_raster:

    # rasterio wants the axis in the order bands, cols, rows.
    new_raster.write(reshape_as_raster(image))

\- wowowo hold your horses, what is this affine, why multiply `*` !?

\- It is just a way to define the upper left origin of our raster and the pixel size! it's super simple, you just needs to use a little bit of **linear algebra**

\* _leaves_


In [ ]:
print(transform)

\* Disclaimer: Sorry to all the people who knows linear algebra. I'm trying to do my best and be short

The `transform` variable is an `Affine`  object that represents an affine transformation like:


$$
\begin{vmatrix}
    x' \\
    y' \\
    1
\end{vmatrix}
= 
\begin{vmatrix}
    a & b & c \\
    d & e & f \\
    0 & 0 & 1
\end{vmatrix}
\ 
\begin{vmatrix}
    x \\
    y \\
    1
\end{vmatrix}
$$

Here with the matrix $\begin{vmatrix} a & b & c \\ d & e & f \\ 0 & 0 & 1\end{vmatrix}$ we transform the image coordinates $x, y$ to world coordinates $x', y'$

It is quite interesnting how this works but we don't have time. Look into [this](https://pygis.io/docs/d_affine.html) for a slightly deeper explaination or this [videos](https://www.youtube.com/watch?v=fNk_zzaMoSs&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab) if you want to get a full intuition on what is going on.

In our application most of the time we will only need `translate` (move the points) and to `scale`

so in the line 
```python
transform = Affine.translation(2.944591, 39.445561) * Affine.scale(xres, yres)
```
we are saying that our 0,0 pixel (in image coordinates, upper left corner) is at  39.445561°N 2.944591°E and the pixels measure 0.005° each side.

_**NOTE** that the units are in degrees because we are using a coordinate reference system that is in degrees!_

So the interesnting bits of our transform matrix are:

$$
\begin{vmatrix}
    x_{size} & \cdot & x_0 \\
    \cdot & y_{size} & y_0 \\
    \cdot & \cdot & \cdot
\end{vmatrix}
$$

this is why the affine object looks like:

```
| 0.005,     0,  2.94|
|     0,-0.005, 39.45|
|     0,     0,     1|
```

## Reprojecting

Now let's reproject a raster from one CRS to another.

But first let's display it..

In [ ]:
with rio.open("../examples/cabrera_rgba.tif") as src:
    img = src.read()
    img_crs = src.crs
    img_extent = rio.plot.plotting_extent(src)
    print(src.transform)

In [ ]:
print(img_extent)
print(img_crs)

In [ ]:
img.shape

In [ ]:
reshape_as_image(img).shape

In [ ]:
plt.imshow(reshape_as_image(img))
plt.gca().set_axis_off()

The crs EPSG:32631 is the UTM zone 31, so if we want to display it we should use a map with the same crs or reproject the raster if we want another one.

For making simple maps we are going to use matplotlib + [Cartopy](https://github.com/SciTools/cartopy)

In [ ]:
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

Cartopy "patches" matplotlib and allow us to define a projection to the axis

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.Mercator())

x0, x1, y0, y1 = img_extent
room = 10_000
plot_extent = x0 - room, x1 + room, y0 - room, y1 + room 
ax.set_extent(plot_extent, crs=ccrs.UTM("31"))
ax.imshow(reshape_as_image(img), transform=ccrs.UTM("31"), extent=img_extent)
# overlay coastline 
ax.add_feature(cartopy.feature.COASTLINE)

_NOTE the coastline resolution is too low for this zoom level. We will fix this in a while._

Reproject to an Orthographic projection with

```python
projection=ccrs.Orthographic(central_longitude=-50, central_latitude=50)
```

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=-50, central_latitude=50))

x0, x1, y0, y1 = img_extent
margin = 10_000

plot_extent = (
    x0 - margin,
    x1 + margin,
    y0 - margin,
    y1 + margin
)

ax.set_extent(plot_extent, crs=ccrs.UTM("31"))
gl = ax.gridlines(draw_labels=True) 
gl.top_labels = gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

ax.imshow(reshape_as_image(img), transform=ccrs.UTM("31"), extent=img_extent)
ax.add_feature(cartopy.feature.COASTLINE)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=-50, central_latitude=50))
ax.coastlines()
gl = ax.gridlines(draw_labels=True)
gl.top_labels = gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

### Changing CRS of the data

For now we have only been changing the projection on the fly for the map.


What if we want to do some analysis or store the data in another CRS abd make it persistent?

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:4326'  # geographical coordinate system that uses lat long degrees

with rio.open('../examples/cabrera_rgba.tif') as src:  # open the original image
    # compute the new affine transform, width and height of the raster 
    # in the new crs.
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds, resolution=0.001)
    dst_kwargs = src.meta.copy()
    dst_kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    with rio.open('../data/cabrera_rgba_reproj.tif', 'w', **dst_kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

In [ ]:
with rio.open("../data/cabrera_rgba_reproj.tif") as src:
    print(src.crs)
    print(src.transform)

Most of the magic hapens in `calculate_default_transform` where we calculate the new Affine matrix for the destination crs.

Then `reproject` deals with witring the new data with the correct metadata.

## Xarray + Rasterio!

In [ ]:
import xarray as xr

Xarray is, in short, a multidimensional labeled array. 

Think it like a mix between pandas with its labeled axes and the flexible arbitrary number of dimensions of ndarray.

In [ ]:
data = xr.DataArray(
    np.random.randint(0, 255, (128, 128, 3)),
    coords={"x": np.arange(128), "y": np.arange(128)},
    dims=["x", "y", "band"]
)

In [ ]:
data

In [ ]:
data.plot.imshow()

In [ ]:
data.sel(band=2).plot.imshow()

In [ ]:
import rioxarray

In [ ]:
raster = rioxarray.open_rasterio("../examples/cabrera_rgba.tif")

In [ ]:
raster

In [ ]:
# to make the image square we need to set the aspect ratio
raster.plot.imshow(size=5, aspect=raster.x.size / raster.y.size)

In [ ]:
raster.plot.imshow(col="band", cmap="gray")

In [ ]:
import geopandas as gpd

In [ ]:
coast = gpd.read_file("../examples/costa_balears.geojson")
coast.head()

In [ ]:
coast.plot()
print(coast.crs)

Let's remove the sea by clipping it out with the help of the coastline polygon (and rioxarray).

In [ ]:
clipped = raster.rio.clip(coast.geometry.to_list(), crs=coast.crs)

In [ ]:
clipped.plot.imshow(size=5, aspect=clipped.x.size / clipped.y.size)
plt.gca().set_axis_off()

How about adding it to an interactive map!?

First of all we will need to convert it to lat long since `ImageOverlay` wants lat long data:

In [ ]:
clipped = clipped.rio.reproject("epsg:4326")

In [ ]:
clipped.rio.transform()

In [ ]:
import folium
from utils import convert_bounds

In [ ]:
m = folium.Map([39.1, 2.97], zoom_start=11)
folium.raster_layers.ImageOverlay(
    image=reshape_as_image(clipped.values),
    bounds=convert_bounds(clipped.rio.bounds()),
).add_to(m)

folium.LayerControl().add_to(m)
m

we can also add vector overlays directly from geopandas!

In [ ]:
m = folium.Map([39.1, 2.97], zoom_start=11)
# geopandas datafame .explore accepts an existing leafle map instance
coast.explore(m=m)
folium.raster_layers.ImageOverlay(
    image=reshape_as_image(clipped.values),
    bounds=convert_bounds(clipped.rio.bounds()),
).add_to(m)
folium.LayerControl().add_to(m)
m